In [1]:
# ImageDataGenerator kütüphanesi resimleri toplu olarak değil de sırayla yükleyerek belleğin daha iyi çalışmasını sağlar.

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
# Versiyon kontrol edilir.

tf.__version__

'2.9.1'

In [3]:
# Eğitim setinin ön işlenmesi

train_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)

training_set = train_datagen.flow_from_directory('../dataset/training_set',
                                                target_size = (64,64), #hedef boyutu 64x64 piksel
                                                batch_size = 32, #bu değerin büyük olması daha doğru gradient değeri hesaplamasını sağlar
                                                class_mode = 'binary')

Found 8500 images belonging to 2 classes.


In [4]:
# Test setinin ön işlenmesi

test_datagen = ImageDataGenerator(rescale = 1./255)

test_set = test_datagen.flow_from_directory('../dataset/test_set',
                                                target_size = (64,64),
                                                batch_size = 32,
                                                class_mode = 'binary')

Found 1000 images belonging to 2 classes.


In [5]:
# CNN mimarisi oluşturuluyor

cnn = tf.keras.models.Sequential()

In [6]:
# Adım 1 : Convolution

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64,64,3]))

# Gizli katmanlar arasında relu aktivasyon fonksiyonu kullanılıyor.
# Conv2D fonksiyonu sayesinde datalar 64x64 boyutunda ele alınacak.
# input_shape deki 3 parametresi ise resimler renkli olduğu için 3 adet katman oluşturulacak.
# Bu katmanlar kırmızı, yeşil ve mavi.

In [7]:
# Adım 2 : Pooling

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Pooling işlemi için MaxPool yöntemi kullanılacaktır.
# strides yani adım sayısı 2 olarak belirlenmiştir.

In [8]:
# Adım 3 : İkinci Convolution Katmanı ve İkinci Pooling İşlemi

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu')) # Convolution
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2)) # Pooling

In [9]:
# Adım 4 : Flattening

cnn.add(tf.keras.layers.Flatten())

# Gelen veriler üzerinde flatting işlemi yapılıyor
# Bu, verilerin yapay sinir ağına gelmeden hemen önce düzleştirilmesidir.

In [10]:
# Adım 5 : Full Connection

cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Burda NN deki gizli katmanlar ve nöronlar oluşturulur.
# Kaç katman olacağı units metodu ile 128 olarak belirlenmiştir. Sebebi resimlerin 64x64 boyutunda olmasıdır.

In [11]:
# Adım 6 : Çıkış Katmanı

cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Çıkış katmanında sınıflandırma yapılacağı için sigmoid kullanılmıştır.
# Çıktı 1 olarak çıkarsa dog, 0 olarak çıkarsa cat dir.

In [12]:
# Compile (Derleme) İşlemi

cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
# Eğitim işlemi

cnn.fit(x=training_set, validation_data=test_set, epochs=45)

# epochs değeri ile 45 adımda öğrenme işlemi yapacak.
# Bu sayıdan başka sayılar da denenmiş ve accuary değerinin 1'e en yakın olduğu sonuç epochs=50 ile bulunmuştur.
# Daha fazla olduğunda ise ezberleme yöntemine gittiği görülmüştür.
# Aktivasyon fonksiyonunun katsayılarını her seferinde güncelleyecek.

# Eğitim sırasından accuracy değeri düşük başlayacaktır yaklaşık %50 gibi
# Veri seti incelendikçe ve loss fonksiyonu ile tekrar katsayıları güncelledikçe accuracy değeri yükselecektir.

Epoch 1/45
266/266 [==============================] - 79s 291ms/step - loss: 0.6178 - accuracy: 0.6439 - val_loss: 0.7364 - val_accuracy: 0.6510
Epoch 2/45
266/266 [==============================] - 85s 319ms/step - loss: 0.5216 - accuracy: 0.7411 - val_loss: 0.5254 - val_accuracy: 0.7390
Epoch 3/45
266/266 [==============================] - 81s 306ms/step - loss: 0.4867 - accuracy: 0.7651 - val_loss: 0.4903 - val_accuracy: 0.7670
Epoch 4/45
266/266 [==============================] - 69s 261ms/step - loss: 0.4562 - accuracy: 0.7816 - val_loss: 0.4603 - val_accuracy: 0.7810
Epoch 5/45
266/266 [==============================] - 73s 276ms/step - loss: 0.4402 - accuracy: 0.7952 - val_loss: 0.5094 - val_accuracy: 0.7640
Epoch 6/45
266/266 [==============================] - 84s 317ms/step - loss: 0.4243 - accuracy: 0.7973 - val_loss: 0.4861 - val_accuracy: 0.7830
Epoch 7/45
266/266 [==============================] - 90s 339ms/step - loss: 0.4180 - accuracy: 0.8042 - val_loss: 0.4975 - val_ac

In [98]:
# Tahmin Oluşturma

import keras
import tensorflow as tf
import numpy as np
from keras.preprocessing import image

# Aşağıdaki single_prediction/ altına atılan resimler manuel olarak test edilebilir, isimlendirmeyi değiştirmek yeterlidir.

test_image = keras.utils.load_img('../dataset/single_prediction/8.jpg', target_size=(64, 64))
test_image = keras.utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

result = cnn.predict(test_image)
training_set.class_indices
if result[0][0]==1:
    prediction = 'dog'
else:
    prediction = 'cat'

1/1 [==============================] - 0s 30ms/step


In [99]:
# Sonuç

print(prediction)

cat
